# Citibike - Statistical Analysis Project

August 1, 2020

<u>About the Data:</u>   https://www.citibikenyc.com/system-data <br>
<u>Data Source:</u>    https://s3.amazonaws.com/tripdata/index.html<br>
<u>Data Date Range:</u>    June 2019 - June 2020

## About the Data

<u>Data Columns:</u>
- Trip Duration (seconds)
- Start Time and Date
- Stop Time and Date
- Start Station Name
- End Station Name
- Station ID
- Station Lat/Long
- Bike ID
- User Type (Customer = 24-hour pass or 3-day pass user; Subscriber = Annual Member)
- Gender (Zero=unknown; 1=male; 2=female)
- Year of Birth

<u>Additional Notes:</u>
- Test trips & trips < 60 seconds (potentially false starts or users tyring to redock bike) have been removed.
- Milage estimates are calculated using an assumed speed of 7.456 miles per hour, up to two hours. Trips over two hours max-out at 14.9 miles. Once you opt into Ride Insights, the Citi Bike app will use your phone's location to record the route you take between your starting and ending Citi Bike station to give exact mileage.
- We only include trips that begin at publicly available stations (thereby excluding trips that originate at our depots for rebalancing or maintenance purposes).

## Setup

### Import libraries

In [1]:
# import relevant libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import pickle
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option('display.float_format', lambda x: '%f' % x)

C:\Users\Grace\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Import data

In [2]:
os.getcwd()

'C:\\Users\\Grace\\Documents\\My_Git_Repos\\citibike-analysis'

In [3]:
os.chdir("data/2019") 

In [4]:
os.listdir()

['201901-citibike-tripdata.csv',
 '201902-citibike-tripdata.csv',
 '201903-citibike-tripdata.csv',
 '201904-citibike-tripdata.csv',
 '201905-citibike-tripdata.csv',
 '201906-citibike-tripdata.csv']

In [5]:
# import all 2019 files into a list of dataframes
dfs_2019 = []
for file in os.listdir():
    dfs_2019.append(pd.read_csv(file))

In [6]:
# concatenate all dataframes into a single dataframe
df_2019 =pd.concat(dfs_2019,ignore_index=True)

In [7]:
df_2019.shape

(9055018, 15)

In [8]:
# pickle the dataframe for easy re-import
df_2019.to_pickle('df_2019.pickle')

In [13]:
os.chdir("..") 

In [15]:
os.getcwd()

'C:\\Users\\Grace\\Documents\\My_Git_Repos\\citibike-analysis\\data'

In [16]:
os.chdir('2020')

In [17]:
os.listdir()

['202001-citibike-tripdata.csv',
 '202002-citibike-tripdata.csv',
 '202003-citibike-tripdata.csv',
 '202004-citibike-tripdata.csv',
 '202005-citibike-tripdata.csv',
 '202006-citibike-tripdata.csv']

In [18]:
# import all 2020 files into a list of dataframes
dfs_2020 = []
for file in os.listdir():
    dfs_2020.append(pd.read_csv(file))

In [19]:
# concatenate all dataframes into a single dataframe
df_2020 =pd.concat(dfs_2020,ignore_index=True)

In [20]:
df_2020.shape

(7508808, 15)

In [21]:
# pickle the dataframe for easy re-import
df_2020.to_pickle('df_2020.pickle')

## Data QA

### Set datatypes

Check that you have the right datatypes.

In [22]:
# original datatypes
df_2019.dtypes

tripduration                 int64
starttime                   object
stoptime                    object
start station id           float64
start station name          object
start station latitude     float64
start station longitude    float64
end station id             float64
end station name            object
end station latitude       float64
end station longitude      float64
bikeid                       int64
usertype                    object
birth year                   int64
gender                       int64
dtype: object

In [23]:
df_2020.dtypes

tripduration                 int64
starttime                   object
stoptime                    object
start station id             int64
start station name          object
start station latitude     float64
start station longitude    float64
end station id               int64
end station name            object
end station latitude       float64
end station longitude      float64
bikeid                       int64
usertype                    object
birth year                   int64
gender                       int64
dtype: object

`starttime` and `stoptime` are current set as strings so converting them to datetime.

In [24]:
df_2019['starttime'] = df_2019['starttime'].astype('datetime64')
df_2019['stoptime'] =df_2019['stoptime'].astype('datetime64')

df_2020['starttime'] = df_2020['starttime'].astype('datetime64')
df_2020['stoptime'] =df_2020['stoptime'].astype('datetime64')

### Create additional features


<i>New features:</i>
1. `tripduration_min`
2. `starttime_year`
3. `starttime_month`
4. `age`

Since `tripduration` is in seconds, let's create another column that shows duration in minutes.

In [18]:
df_2019['tripduration_min'] = df_2019['tripduration']/60
df_2020['tripduration_min'] = df_2020['tripduration']/60

It may also be helpful to extract the year & month of each starttimes.

In [ ]:
# 2019 data
df_2019['starttime_year'] = pd.DatetimeIndex(df_2019['starttime']).year
df_2019['starttime_month'] = pd.DatetimeIndex(df_2019['starttime']).month

# 2020 data
df_2020['starttime_year'] = pd.DatetimeIndex(df_2020['starttime']).year
df_2020['starttime_month'] = pd.DatetimeIndex(df_2020['starttime']).month

Create `age` from `birth year`.

In [30]:
df_2019['age'] = 2019 - df_2019['birth year']
df_2020['age'] = 2020 - df_2020['birth year']

In [31]:
df_2019[['age', 'birth year']].head()

,age,birth year
0,48,1971
1,55,1964
2,32,1987
3,29,1990
4,40,1979


In [32]:
df_2020[['age', 'birth year']].head()

,age,birth year
0,28,1992
1,51,1969
2,57,1963
3,40,1980
4,38,1982


## Data Analysis

###  Questions

- Ridership numbers
- Duration of trips
- Popular & least popular stations (start & end stations)
- Popular routes (maybe visualize the routes on a map)
- Who's riding Citibikes during COVID (age, gender)?
- How has bike rental behavior changed due to COVID?
- Did we lose subscribers during COVID?

### Ridership Numbers

In [23]:
big_df.isna().sum()

tripduration                 0
starttime                    0
stoptime                     0
start station id           143
start station name         143
start station latitude       0
start station longitude      0
end station id             143
end station name           143
end station latitude         0
end station longitude        0
bikeid                       0
usertype                     0
birth year                   0
gender                       0
tripduration_min             0
starttime_year               0
starttime_month              0
dtype: int64

In [21]:
big_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,tripduration_min,starttime_year,starttime_month
0,330,2019-06-01 00:00:01.500,2019-06-01 00:05:31.760,3602.000000,31 Ave & 34 St,40.763154,-73.920827,3570.000000,35 Ave & 37 St,40.755733,-73.923661,20348,Subscriber,1992,1,5.500000,2019,6
1,830,2019-06-01 00:00:04.240,2019-06-01 00:13:55.147,3054.000000,Greene Ave & Throop Ave,40.689493,-73.942061,3781.000000,Greene Av & Myrtle Av,40.698568,-73.918877,34007,Subscriber,1987,2,13.833333,2019,6
2,380,2019-06-01 00:00:06.019,2019-06-01 00:06:26.779,229.000000,Great Jones St,40.727434,-73.993790,326.000000,E 11 St & 1 Ave,40.729538,-73.984267,20587,Subscriber,1990,2,6.333333,2019,6
3,1155,2019-06-01 00:00:06.776,2019-06-01 00:19:22.538,3771.000000,McKibbin St & Bogart St,40.706237,-73.933871,3016.000000,Kent Ave & N 7 St,40.720368,-73.961651,33762,Subscriber,1987,1,19.250000,2019,6
4,1055,2019-06-01 00:00:07.520,2019-06-01 00:17:42.558,441.000000,E 52 St & 2 Ave,40.756014,-73.967416,3159.000000,W 67 St & Broadway,40.774925,-73.982666,31290,Subscriber,1973,1,17.583333,2019,6
